# Logistic regression from a neural network perspectives

In this notebook we will see how to implement logistic regression from a neural network perspective using the `python` library `Keras` (https://keras.io/)

### Prerequisites

Some concepts are assumed to be known (at least partially) and will not be covered:

- splitting the data in the **training**, **validation** and **test** sets (cross-validation and overfitting)
- binary classification problems and logistic regression
- basic Python

However, the material can be used withouth knowing the above.

## Keras
https://drive.google.com/file/d/1KeEVvLRZztK5tEQG0wmhRAiFumlD2kmo/view?usp=sharing
<img src="https://drive.google.com/uc?export=view&id=1KeEVvLRZztK5tEQG0wmhRAiFumlD2kmo">

- Large community of users
- Multi-backend, multi-platform
- Easy and quick development and deployment of deep learning models



### Steps in a Keras implementation of deep learning

1. **set up**: import libraries and configure the environment (e.g. set seed)
2. **data management**: load data and preprocessing, training and test sets
3. **define hyperparameters**: n. of nodes, n. of layers, type of layers, activation function, loss function, optimiser, n. of epochs (deep learning architecture)
4. **build the model**: define the topology of the network and build the stack of layers
5. **compile the model**: compile the model by combining the defined layers, the loss function, the optimiser and the chosen metrics for evaluation
6. **train the model**: fit the deep learning model for the chosen n. of epochs (iterations)
7. **predictions**: use the trained model to obtain predictions
8. **evaluate the model**: use test data to measure how well the model did (loss, accuracy metrics)
9. **'rinse and repeat'**: modify and repeat until satisfied (e.g. change hyperparameters, model architecture, network topology etc.)

## Set up

Libraries

In [ ]:
## import libraries
import numpy as np
import tensorflow
import pandas as pd
import sklearn.datasets
import matplotlib.pyplot as plt

## The data

The dataset we are going to use is very famous. It was published by Ronald Fisher in 1936 together with the paper [The use of multiple measurements in taxonomic problems](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1469-1809.1936.tb02137.x). Data are public and nowadays this dataset is shipped with many statistical software and packages. We are going to use the copy coming with [sci-kit learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

In [ ]:
iris = sklearn.datasets.load_iris()
iris.data = pd.DataFrame(iris.data, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(iris.target) #converting numpy array -> pandas Series
iris.target = iris.target.to_frame() #converting Pandas series to dataframe
print('Shape of the feature table: ' + str(iris.data.shape))
print('Shape of the target array: ' + str(iris.target.shape))

In [ ]:
print(iris.data)

In [ ]:
print(iris.target)

In [ ]:
features = iris.data.iloc[:,:]
target = iris.target

#change these two values to plot different features, remembering the numbering:
# 0 : sepal length (cm)
# 1 : sepal width (cm)
# 2 : petal length (cm)
# 3 : petal width (cm)
feature_x = 0
feature_y = 1

#starting a new plot
fig, ax = plt.subplots()

#adding data in two bunches
ax.scatter(x=iris.data.iloc[0:50,feature_x],    y=iris.data.iloc[0:50,feature_y],    c='red',   label=iris.target_names[0])
ax.scatter(x=iris.data.iloc[50:100,feature_x],  y=iris.data.iloc[50:100,feature_y],  c='green', label=iris.target_names[1])
ax.scatter(x=iris.data.iloc[100:150,feature_x], y=iris.data.iloc[100:150,feature_y], c='blue',  label=iris.target_names[2])

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

We are now using all four features of the `iris` dataset: `sepal length (cm)`, `sepal width (cm)`, `petal length (cm)` and `petal width (cm)`.
We first reduce the problem to binary classification, by merging two classes together

In [ ]:
#updating class labels. To makes things difficult we put together old classes 0 and 1
#in a new class (non virginica) and keep old class 2 (virginica) as new class 1.
#For an easier problems put together versicolor and virginica and keep setosa by itself
n1 = 100 ## split: 50 for setosa vs versicolor+virginica, 100 for setos+versicolor vs virginica
target[0:n1] = 0 
target[n1:150] = 1

print(iris.target.iloc[:,0].value_counts())

As before, a visual check of the two classes:

In [ ]:
#starting a new plot
fig, ax = plt.subplots()

#adding data in two bunches
ax.scatter(x=features.iloc[0:n1,0],   y=features.iloc[0:n1,1],   c='red',  label='Non virginica')
ax.scatter(x=features.iloc[n1:150,0], y=features.iloc[n1:150,1], c='blue', label='Virginica')

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

### Neural Network model

In the slides we saw first how to implement logistic regression as a simple neural network model with just the output layer: this output layer had only one node (binary classification) which performed both the regression and sigmoid activation steps:

<img src="https://drive.google.com/uc?export=view&id=1PRc719uT1kOUuCMbpHML2sEk7qp6UJnm">

We are now building a **shallow neural network model**, by adding **one hidden layer** with **u nodes** (units):

<img src="https://drive.google.com/uc?export=view&id=1QROz9pFnMoqTeqrFbele8pFz8qXDSckq">

#### Training and test sets

We first prepare the training and test set for correct evaulation of the neural network model performance: we make one split (one training set and one test set), and assign 80% of the data to the training set and 20% of the data to the test set

In [ ]:
#we want to have the same proportion of classes in both train and validation sets
from sklearn.model_selection import StratifiedShuffleSplit

#building a StratifiedShuffleSplit object (sss among friends) with 20% data
#assigned to validation set (here called "test")
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#the .split() method returns (an iterable over) two lists which can be
#used to index the samples that go into train and validation sets
for train_index, val_index in sss.split(features, target):
    features_train = features.iloc[train_index, :]
    features_val   = features.iloc[val_index, :]
    target_train   = target.iloc[train_index,:]
    target_val     = target.iloc[val_index,:]
    
#let's print some shapes to get an idea of the resulting data structure
print("Training features size: ", features_train.shape)
print("Test features size: ", features_val.shape)
print("Training targets size: ", target_train.shape)
print("Test targets size: ", target_val.shape)

print("Type of the training features object: ", type(features_train))
print("Type of the training targets object: ", type(target_train))

In [ ]:
print(target_train.iloc[:,0].value_counts())

In [ ]:
print(target_val.iloc[:,0].value_counts())

## Shallow neural network models

#### Building the neural network model

We now build the (shallow) **neural network model** using the `Keras` framework: we choose a `sequential` architecture and add `dense` (fully connected) layers (**1 hidden layer**, **1 output layer**).

We have to set a number of `hyperparameters` (the **building blocks** -or 'ingredients'- of a neural network model):

- the **number of hidden nodes** (number of units in the hidden layer)
- the **type of activation function** in the hidden layer
- the **output activation function**
- the **loss function** (for backpropagation)
- the **optimizer** (for gradient descent)

In [ ]:
## # Configuration options
input_shape = (features.shape[1],) ## tuple that specifies the number of features 
hidden_nodes = 8
hidden_activation = 'relu'
output_activation = 'sigmoid'
loss_function = 'binary_crossentropy'
optimizer_used = 'SGD' ##stochastic gradient descent
num_epochs = 200

We chose `ReLU` activation functions for the units in the hidden layer, and `sigmoid` activation function for the output layer.

We chose <a href='https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a'>binary cross-entropy</a> for binary classification problems: this is the loss function we have been discussing in the slides.
As for the optimizer, we use `SDG`: more details on the available otimizers in Keras can be found <a href='https://keras.io/api/optimizers/'>here</a>

In [ ]:
print(input_shape)

### Binary classification

<!-- We revise here the classification exercises on the `iris` dataset moving from logistic regression to shallow neural networks. -->
Initially, we'll address binary classification. 

We first import some libraries and the dataset:

In [ ]:
#we are building a "sequential" model, meaning that the data will 
#flow like INPUT -> ELABORATION -> OUTPUT.
from keras.models import Sequential

#a "dense" layer is a layer were all the data coming in are connected
#to all nodes.
from keras.layers import Dense

# binary classification shallow neural network model in Keras
model = Sequential()
model.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model.add(Dense(1, activation=output_activation))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizer_used, loss=loss_function)

In [ ]:
print(model.summary())

The `summary()` method of the Keras model tells us that there are 49  parameters to train:
- w1, w2, w3, w4, b (weights for the 4 features + bias term) for each of the 8 nodes in the hidden layer ($\rightarrow$ 40 parameters);
- w1 - w8 + b (weights for the 8 nodes results + bias term) for the output layer ($\rightarrow$ 9 parameters) [&#161; you may remember from the matrix dimensions we discussed in the slides !]

#### Training the neural network model

We have now prepared everything we need and are ready to train the model on our data. It's an iterative process that cycles many times through what are called `epochs` (~ iterations). We'll start with using the parameter set above:


In [ ]:
history = model.fit(features_train, target_train, epochs=num_epochs, validation_data=(features_val, target_val))

In [ ]:
def plot_loss_history(h, title):
    plt.plot(h.history['loss'], label = "Train loss")
    plt.plot(h.history['val_loss'], label = "Validation loss")
    plt.xlabel('Epochs')
    plt.title(title)
    plt.legend()
    plt.show()

plot_loss_history(history, 'Logistic ({} epochs)'.format(num_epochs))

#### Confusion matrix

With more than two features we can't plot the decision boundary; however, we can have an idea of the performance of our classification model by looking at the <a href="https://en.wikipedia.org/wiki/Confusion_matrix">confusion matrix</a>:

In [ ]:
history.history.keys()

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = model.predict(features_val)
print(predictions)
predicted_labels = np.where(predictions > 0.5, "virginica", "non-virginica")
target_labels = target_val.to_numpy().reshape((len(target_val),1))
target_labels = np.where(target_labels > 0.5, "virginica", "non-virginica")
print(target_labels)
con_mat_df = confusion_matrix(target_labels, predicted_labels, labels=["non-virginica","virginica"])
print(con_mat_df)

In [ ]:
import seaborn as sn

figure = plt.figure(figsize=(8, 8))
sn.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

We can now calculate the **overall accuracy** of the model, and the **TPR** (true positive rate) and the **TNR** (true negative rate):

$$
\text{TPR}=\frac{TP}{TP+FN}\\
$$

$$
\text{TNR}=\frac{TN}{TN+FP}\\
$$


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(target_labels, predicted_labels)
tn, fp, fn, tp = con_mat_df.ravel()
tpr = tp/(tp+fn)
tnr = tn/(tn+fp)

print("Overall accuracy is: ", accuracy)
print("TPR is: ", tpr)
print("TNR is: ", tnr)

- going deeper
- DL for logistic regression is a bit of an overkill (Ng's slide below)

## Multiclass classification

We now extend logistic regression to multiclas classification using [softmax](https://en.wikipedia.org/wiki/Softmax_function).

The problem requires us to do three-classes classification using a Softmax function, which can be easily considered as an extension of logistic regression over three (or more) classes. We use the neural network-like implementation as with binary classification.

Luckily, Keras provides a [softmax activation function](https://keras.io/api/layers/activations/#softmax-function), which we will use instead of the logistic we previously used.

The structure of our network will be similar, but the output goes from a single number to **three** numbers, one per class, and we thus need three nodes:

<img src="https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/data/softmax_neuron.png">

As a result, the loss function will need to change. Remember, loss represents a measure of how good the predictions are. Previously we used binary_crossentropy, but since now predictions are multiclass we need to change function. Luckily Keras provides a natural extension for the multiclass case with [CategoricalCrossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class)



The problem is that our target array `iris.target` is a numeric array. But those numbers we used (0, 1, and 2) do not represent real values. In other words, "virginica" is not twice "versicolor". Numbers here are used as labels, not as quantities.

In fact, to properly train a model the structure of the target array must change to [one-hot encoding](https://en.wikipedia.org/wiki/One-hot). In simple terms, it needs to become a table with one row per sample (150 in total) and one column per class (three in total). Something like:

| Setosa | Versicolor | Virginica |
|------|------|------|
|   0  |   1  |   0  |
|   1  |   0  |   0  |
|   1  |   0  |   0  |
|   0  |   0  |   1  |

As you can see the first sample is Versicolor, the second and third are Setosa, the last one is Virginica. Note that there is only a single "one" per row.

Luckily, it's easy to pass to one-hot encoding using keras function [to_categorical](https://keras.io/api/utils/python_utils/#to_categorical-function):

In [ ]:
iris = sklearn.datasets.load_iris()
iris.data = pd.DataFrame(iris.data, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(iris.target) #converting numpy array -> pandas Series
iris.target = iris.target.to_frame() 

features = iris.data.iloc[:,:]
target = iris.target

#the "utils" subpackage is very useful, take a look to it when you have time
from tensorflow.keras.utils import to_categorical

#converting to categorical
target_multi_cat = to_categorical(target)

#since everything else is a Pandas dataframe, let's stick to the format
#for consistency
target_multi_cat = pd.DataFrame(target_multi_cat)

#let's take a look
print(target_multi_cat)

### Training and validation sets

We are now ready to create our training and validation sets, as done above:

In [ ]:
#we want to have the same proportion of classes in both train and validation sets
from sklearn.model_selection import StratifiedShuffleSplit

test_pct = 0.2

#building a StratifiedShuffleSplit object (sss among friends) with 20% data
#assigned to validation set (here called "test")
#random_state is used to control class balance between training and test sets (None to switch to random behavior)
sss = StratifiedShuffleSplit(n_splits=1, test_size= test_pct, random_state=0)

for train_index, val_index in sss.split(features, target_multi_cat):
    features_train = features.iloc[train_index, :]
    features_val   = features.iloc[val_index, :]
    target_train   = target_multi_cat.iloc[train_index, :]
    target_val     = target_multi_cat.iloc[val_index, :]

Just a quick check:

In [ ]:
#shapes
print(features_train.shape)
print(features_val.shape)
print(target_train.shape)
print(target_val.shape)

#number of classes per split
print('\nClasses in train set:')
print(target_train.sum())
print('\nClasses in validation set:')
print(target_val.sum())

We have now a balanced dataset, with 40 instances for each class in the training set and 10 in the validation set.

### Set up

We define here the hyperparameters of the model

In [ ]:
## # Configuration options
num_classes = 3
input_shape = (features.shape[1],)
hidden_nodes = 8
hidden_activation = 'relu'
output_activation = 'softmax'
loss_function = 'categorical_crossentropy'
optimizer_used = 'rmsprop'
num_epochs = 200

### A multiclass model

We are now ready to declare our neural network model for multiclass classification: we use `Keras`. The output layer has three units, corresponding to the three classes:

In [ ]:
#we are building a "sequential" model, meaning that the data will 
#flow like INPUT -> ELABORATION -> OUTPUT.
from keras.models import Sequential

#a "dense" layer is a layer were all the data coming in are connected
#to all nodes.
from keras.layers import Dense
# 3-class softmax regression in Keras
model_multi = Sequential()
model_multi.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model_multi.add(Dense(num_classes, activation=output_activation))

#compile the model specifying the new multiclass loss
model_multi.compile(optimizer=optimizer_used, loss=loss_function)

In [ ]:
print(model_multi.summary())

We see that we have 67 parameters to train: 5 parameters (w1 ... w4, b) times 8 nodes in the hidden layer; 9 parameters (8 units + b) times three nodes in the output layer.

In [ ]:
history_multi = model_multi.fit(features_train, target_train, epochs=num_epochs, 
                     validation_data=(features_val, target_val), verbose=0)

In [ ]:
plot_loss_history(history_multi, 'Softmax multiclass (200 epochs)')

In [ ]:
predictions = model_multi.predict(features_val)
print(predictions)

# predicted_classes = model_multi.predict_classes(features_val)
predicted_classes = np.argmax(model_multi.predict(features_val), axis=-1)
target_classes = target.iloc[val_index,:].to_numpy()
con_mat_df = confusion_matrix(target_classes, predicted_classes)
print(con_mat_df)

In [ ]:
figure = plt.figure(figsize=(8, 8))
sn.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = accuracy_score(target_classes, predicted_classes)
print("Overall accuracy is: ", accuracy)

confusion_matrix(target_classes, predicted_classes, normalize='true')

## A side note

Neural networks models and deep learning models may be an overkill when applied to a classification problem on 150 samples and 4 features: it is not even guaranteed that they will perform better than simpler machine learning methods like logistic regression. Indeed at times deep learning may perform worse than simpler approaches. 
As a matter of fact, deep learning is known to work best when the size of the problem is very large (both in terms of amount of data and order of computations needed), and the advent of *Big Data* is exactly one of the drivers behind the rise of deep learning.

The Figure below comes from slides by <a href='https://en.wikipedia.org/wiki/Andrew_Ng'>Andrew Ng</a> and illustrates this point: when the amount of data is limited, traditional machine learning methods and small and large neural networks have similar performance. It is only when the size of the problem increases that deep learning shows its potential and consistently outperforms other methods/algorithms.



<img src="https://drive.google.com/uc?export=view&id=1_Nom99R963AhM30UbbDLg4-u-mPSiX_L">

As a matter of fact, a reliable way to get better predictive performance is often to either **train a bigger network** or feed **more data** to it.
Eventually you'll hit the limit: i) run out of training examples; or ii) network so big that it is too slow to train 